In [ ]:
#imports
import os

from google.colab import drive
import numpy as np
import matplotlib as plt
from matplotlib import pyplot
import pandas as pd
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
import gc

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

import numpy as np
import keras
from keras import models, layers, Input, regularizers
from keras.utils import np_utils
from keras.models import Sequential, Model
import matplotlib.pyplot as plt
from keras.layers import Dense
import random
random.seed(42)

from sklearn.utils import class_weight
from sklearn.utils import shuffle

In [ ]:
#loading data from drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/ICASSP-2021/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Toggles that may need to be updated every round are denoted with ###

In [ ]:
test_set = 'extra-online-data/'  ### location of the new data output from the predictions code
training_run = '3'  ### this is equivalent to the quarter number from the predictions code
data_file = '/content/drive/My Drive/ICASSP-2021/'

In [ ]:
train_val = np.load(data_file + test_set + 'extra_train' + training_run + '.npy')
train_val_classes = np.load(data_file + test_set + 'extra_train_labels' + training_run + '.npy') 
test = np.load(data_file + test_set + 'extra_val' + training_run + '.npy') 
test_classes = np.load(data_file + test_set + 'extra_val_labels' + training_run + '.npy') 

In [ ]:
model = keras.models.load_model('/content/drive/My Drive/ICASSP-2021/online-code/models/initial/self.hdf5')

In [ ]:
filepath =  '/content/drive/My Drive/ICASSP-2021/' + test_set 
name = 'healthy-sick-extra' + training_run + '.hdf5' 
auc = tf.keras.metrics.AUC()
calls = tf.keras.callbacks.ModelCheckpoint(
    filepath + name, monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=False
)

In [ ]:
max_value = (np.max(train_val))

In [ ]:
train_val = train_val / max_value

In [ ]:
train_val = np.concatenate((train_val, train_val, train_val), axis = -1)

In [ ]:
train_val_classes2 = []

for i in range(len(train_val_classes)):
    if (train_val_classes[i] == 2):
        train_val_classes[i] = 1
    train_val_classes2.append(train_val_classes[i])

train_val_classes = np.array(train_val_classes2)


In [ ]:
test = test / max_value

In [ ]:
test = np.concatenate((test, test, test), axis = -1)

In [ ]:
test_classes2 = []

for i in range(len(test_classes)):
    if (test_classes[i] == 2):
        test_classes[i] = 1
    test_classes2.append(test_classes[i])

test_classes = np.array(test_classes2)


In [ ]:
test_classes = tf.keras.utils.to_categorical(test_classes)

In [ ]:
train_val, train_val_classes = shuffle(train_val, train_val_classes)

In [ ]:
np.unique(train_val_classes, return_counts=True)

(array([0., 1.]), array([5652, 3622]))

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_val_classes),
                                                 train_val_classes)

In [ ]:
train_val_classes = tf.keras.utils.to_categorical(train_val_classes)

In [ ]:
class_weights[0]

0.8204175513092711

In [ ]:
class_weights = {0: class_weights[0],
                1: class_weights[1]}

In [ ]:
#------------------------------------------

In [ ]:
#augs
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.2,
    channel_shift_range=0.0,
    fill_mode="nearest",
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=False,
    rescale=None,
    preprocessing_function=None,
    data_format=None,
    validation_split=0.0,
    dtype=None,
)

train_generator = train_datagen.flow(train_val, train_val_classes)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator()
test_generator = test_datagen.flow(test, test_classes)

In [ ]:
train_len = len(train_val)
val_len = len(test)

In [ ]:
for layer in model.layers:
  layer.trainable = True

from keras.optimizers import SGD
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001) , loss='categorical_crossentropy', metrics=["accuracy"])

hist2 = model.fit_generator(train_generator, steps_per_epoch=train_len//32, validation_data=test_generator, validation_steps=val_len//32, epochs=30, class_weight=class_weights, callbacks=calls, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/30
289/289 [==============================] - 197s 420ms/step - loss: 0.6496 - accuracy: 0.7722 - val_loss: 0.3648 - val_accuracy: 0.8701

Epoch 00001: val_accuracy improved from -inf to 0.87005, saving model to /content/drive/My Drive/ICASSP-2021/extra-online-data/healthy-sick-extra3.hdf5
Epoch 2/30
289/289 [==============================] - 109s 371ms/step - loss: 0.3515 - accuracy: 0.8745 - val_loss: 0.4894 - val_accuracy: 0.8671

Epoch 00002: val_accuracy did not improve from 0.87005
Epoch 3/30
289/289 [==============================] - 107s 370ms/step - loss: 0.2755 - accuracy: 0.9151 - val_loss: 0.3795 - val_accuracy: 0.8698

Epoch 00003: val_accuracy did not improve from 0.87005
Epoch 4/30
289/289 [==============================] - 105s 365ms/step - loss: 0.2387 - accuracy: 0.9285 - val_loss: 0.3804 - val_accuracy: 0.8707

Epoch 00004:

In [ ]:
model = keras.models.load_model(filepath + name)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

probs = model.predict(test)
y_pred = np.argmax(probs, axis = 1)
y_true = np.argmax(test_classes, axis = 1)

confusion_matrix(y_true, y_pred)

array([[2380,  123],
       [ 324, 1624]])

In [ ]:
from sklearn.metrics import recall_score
recalls = recall_score(y_true, y_pred, average=None)
recalls

array([0.95085897, 0.83367556])

In [ ]:
recalls = np.array(recalls)
np.save('/content/drive/My Drive/ICASSP-2021/online-code/recalls/' + test_set + 'h_recalls' + training_run + '.npy', recalls)

In [ ]:
print(accuracy_score(y_true, y_pred))

0.8995731296337902
